# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

In [27]:
import numpy as np
import pandas as pd
#Import dataframe flights
flights_test = pd.read_csv("flights_test.csv",low_memory=False).drop('Unnamed: 0',axis=1)
flights = pd.read_csv("flights.csv",low_memory=False).drop('Unnamed: 0',axis=1)[list(flights_test.columns)+['arr_delay']]

In [28]:
flights=flights.dropna()
flights_test =flights_test.dropna()

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

In [29]:
def time_machine(df):
    df['months']=df.fl_date.str[5:7].astype(int)
    df['weekday']=pd.to_datetime(df.fl_date).dt.dayofweek
    return df

In [33]:
time_machine(flights_test)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,months,weekday
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,...,SFO,"San Francisco, CA",1810,1945,N,95,1,363,1,2
1,2020-01-01,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,...,SFO,"San Francisco, CA",1150,1320,N,90,1,363,1,2
2,2020-01-01,WN,WN,WN,4598,WN,N7885A,4598,13891,ONT,...,SJC,"San Jose, CA",2020,2130,N,70,1,333,1,2
3,2020-01-01,WN,WN,WN,4761,WN,N551WN,4761,13891,ONT,...,SJC,"San Jose, CA",1340,1455,N,75,1,333,1,2
4,2020-01-01,WN,WN,WN,5162,WN,N968WN,5162,13891,ONT,...,SJC,"San Jose, CA",915,1035,N,80,1,333,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
660551,2020-01-31,DL,DL_CODESHARE,DL,4954,9E,N926XJ,4954,11278,DCA,...,CVG,"Cincinnati, OH",1859,2041,N,102,1,411,1,4
660552,2020-01-31,DL,DL_CODESHARE,DL,4955,9E,N309PQ,4955,11278,DCA,...,CVG,"Cincinnati, OH",1515,1702,N,107,1,411,1,4
660553,2020-01-31,DL,DL_CODESHARE,DL,4956,9E,N324PQ,4956,12478,JFK,...,BTV,"Burlington, VT",2205,2337,N,92,1,266,1,4
660554,2020-01-31,DL,DL_CODESHARE,DL,4957,9E,N132EV,4957,13930,ORD,...,JFK,"New York, NY",1035,1356,N,141,1,740,1,4


In [31]:
time_machine(flights)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,arr_delay,months,weekday
0,2018-10-08,WN,WN,WN,502,WN,N8628A,502,14679,SAN,...,"Denver, CO",810,1125,N,135.0,1.0,853.0,4.0,10,0
1,2018-10-08,WN,WN,WN,1009,WN,N8676A,1009,14679,SAN,...,"Denver, CO",1535,1850,N,135.0,1.0,853.0,7.0,10,0
2,2018-10-08,WN,WN,WN,1977,WN,N932WN,1977,14679,SAN,...,"Denver, CO",1325,1640,N,135.0,1.0,853.0,-2.0,10,0
3,2018-10-08,WN,WN,WN,2132,WN,N926WN,2132,14679,SAN,...,"Denver, CO",1030,1345,N,135.0,1.0,853.0,3.0,10,0
4,2018-10-08,WN,WN,WN,185,WN,N8623F,185,14679,SAN,...,"Newark, NJ",850,1705,N,315.0,1.0,2425.0,0.0,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15927479,2018-10-08,WN,WN,WN,707,WN,N7882B,707,14679,SAN,...,"Dallas, TX",1625,2115,N,170.0,1.0,1182.0,23.0,10,0
15927480,2018-10-08,WN,WN,WN,1772,WN,N7737E,1772,14679,SAN,...,"Dallas, TX",1830,2320,N,170.0,1.0,1182.0,28.0,10,0
15927482,2018-10-08,WN,WN,WN,2362,WN,N928WN,2362,14679,SAN,...,"Dallas, TX",640,1150,N,190.0,1.0,1182.0,-19.0,10,0
15927483,2018-10-08,WN,WN,WN,268,WN,N230WN,268,14679,SAN,...,"Denver, CO",1850,2205,N,135.0,1.0,853.0,-10.0,10,0


### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.